In [1]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
import pathSetting as setting
import pathSetting

from Clust.clust.ML.common.common import p1_integratedDataSaving as p1

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.ingestion.mongo.mongo_client import MongoClient

db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)
mongo_client = MongoClient(ins.CLUSTMetaInfo2)
"""
#pathSetting
from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)


DataMetaPath = "./integratedData.json"
csvDataFileRootDir ='./data/'
scalerRootDir ='./scaler/'
trainModelMetaFilePath ="./model.json"
"""

'\n#pathSetting\nfrom Clust.setting import influx_setting_KETI as ins\nfrom Clust.clust.ingestion.influx import influx_client_v2 as influx_Client\ndb_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)\n\n\nDataMetaPath = "./integratedData.json"\ncsvDataFileRootDir =\'./data/\'\nscalerRootDir =\'./scaler/\'\ntrainModelMetaFilePath ="./model.json"\n'

In [2]:
def integrated_data_meta(data_name, dataInfo, start_time, end_time, clean_param):
    data_info = {}
    data_info["data_name"] = data_name
    data_info["dataInfo"] = dataInfo
    data_info["startTime"] = start_time
    data_info["endTime"] = end_time
    data_info["cleanParam"] = clean_param
    
    return data_info

In [3]:
db_name = 'integrated_data'

In [4]:
# 테스트를 위한 변수 셋팅 및 설정 - 4개의 데이터를 만들어보기 위한 가상 설정
modeList =['trainX', 'trainy', 'testX', 'testy']
for mode in modeList:
    
    trainStartTime = "2016-01-11"
    trainEndTime = "2016-04-15"

    testStartTime = "2021-01-01"
    testEndTim = "2021-02-12"
    trainXDataInfo = [['life_indoor_environment', 'humidityTrain_10min'], ['life_indoor_environment', 'temperatureTrain_10min'], 
                ['weather_outdoor_environment', 'belgiumChieverseAirportTrain_10min']]

    testXDataInfo = [['life_indoor_environment', 'humidityTest_10min'], ['life_indoor_environment', 'temperatureTest_10min'], 
                ['weather_outdoor_environment', 'belgiumChieverseAirportTest_10min']]
    
    # 1. Set Parameter 
    # 1-1 StartTime, EndTime, DataInfo, Integration_freq_sec, cleanParam, processParam

    if mode == 'trainX':
        dataName = 'IntegraionTrainX'
        startTime = trainStartTime
        endTime = trainEndTime
        dataInfo = trainXDataInfo
        integration_freq_sec = 60 * 10 # 10분

    elif mode =='testX':
        dataName = 'IntegraionTestX'
        startTime = testStartTime
        endTime = testEndTim
        dataInfo = testXDataInfo
        integration_freq_sec = 60 * 10 # 10분

    elif mode == 'trainy':
        dataName = 'IntegraionTrainy'
        startTime = trainStartTime
        endTime = trainEndTime
        dataInfo = [['life_indoor_environment', 'applianceEnergyDatasetTrainy_1day']]
        integration_freq_sec = 60 * 60*24 # 10분
    elif mode =='testy':
        dataName = 'IntegraionTesty'
        startTime = testStartTime
        endTime = testEndTim
        dataInfo = [['life_indoor_environment', 'applianceEnergyDatasetTesty_1day']]
        integration_freq_sec = 60 * 60*24


    #cleanParam ="Clean"
    cleanParam = "NoClean"
    processParam = p1.get_process_param(cleanParam)
    
    # 2. 
    datasetName ="ActionPatternData"
    
    # save to influxdb
    ms_name = datasetName + '_' + mode + '_' + cleanParam
    data = p1.getData(pathSetting.db_client, dataInfo, integration_freq_sec, processParam, startTime, endTime)
    db_client.write_db(db_name, ms_name, data)

    # save to mongodb
    collection_name = datasetName
    data_info = integrated_data_meta(ms_name, dataInfo, startTime, endTime, cleanParam)
    mongo_client.insert_document(db_name, collection_name, data_info)

===integrationStart===
===integrationEnd===
========== write success ==========
========== Data Svae Success ==========
===integrationStart===
===integrationEnd===
========== write success ==========
========== Data Svae Success ==========
===integrationStart===
===integrationEnd===
========== write success ==========
========== Data Svae Success ==========
===integrationStart===
===integrationEnd===
========== write success ==========
========== Data Svae Success ==========


In [5]:
data_info

{'data_name': 'ActionPatternData_testy_NoClean',
 'dataInfo': [['life_indoor_environment', 'applianceEnergyDatasetTesty_1day']],
 'startTime': '2021-01-01',
 'endTime': '2021-02-12',
 'cleanParam': 'NoClean',
 '_id': ObjectId('64350f2c606b386886425724')}